## ASR

In [ ]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline


device = "cuda:0"

model_id = "/home/vapa/projects/iskra/models/asr/whisper-large-v3"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch.float16, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch.float16,
    device=device,
)

In [ ]:
sample = "/home/vapa/projects/iskra/received_audio/test.wav"

result = pipe(sample)
print(result["text"])

## LLM

In [33]:
import requests


payload = {
        "model": "/home/vapa/projects/iskra/models/llms/iskra-8b-sp",
        "messages": [
            {"role": "system", "content": "You are a helpful assistant iskra, help users by answering their questions and entertaining them."},
            {"role": "user", "content": "Play some Slim Shady."}
        ]
    }

response = requests.post("http://0.0.0.0:8000/v1/chat/completions", json=payload)
generated_text = response.json()["choices"][0]["message"]["content"]
generated_text   

'<sp_song> Slim Shady'

# TTS

In [47]:
import requests

url = "http://localhost:5002/api/tts"
headers = {
    "text": "Hello, world you freak man sand",
}

response = requests.post(url, headers=headers)

with open("/home/vapa/projects/iskra/data/tts/responses/output.wav", "wb") as f:
    f.write(response.content)